In [ ]:
#!pip3 install -U spacy                

In [ ]:
import random, collections, re
from wetsuite.datacollect import db
from wetsuite.datacollect import db
import wetsuite.helpers.spacy
import spacy

spacy.require_gpu()
english = spacy.load("en_core_web_trf")  

conn = db.connect()
curs = conn.cursor()
curs.execute('SELECT plaintext FROM bwb LIMIT 100')
bwb_rows = curs.fetchall()
conn.rollback()

In [4]:
import spacy

nlp = spacy.blank("en")  # "Build upon the spaCy Small Model" (seems wrong?)

ruler = nlp.add_pipe("entity_ruler") # adds EntityRuler to pipelines and returns reference

patterns = [
    {"label": "GPE", "pattern": "Treblinka"},
    {"label": "ORG", "pattern": "Wikipedia"},
]

ruler.add_patterns(patterns)

# test on example sentence
text = "Treblinka is a small village in Poland. Wikipedia notes that Treblinka is not large."
doc = nlp(text)
for ent in doc.ents:
    print (ent.text, ent.label_)

Treblinka GPE
Wikipedia ORG
Treblinka GPE


In [9]:
from spacy.tokens import Doc
from spacy.training import Example

predicted = Doc(nlp.vocab, words=["Apply", "some", "sunscreen"])
token_ref = ["Apply", "some", "sun", "screen"]
tags_ref = ["VERB", "DET", "NOUN", "NOUN"]
example = Example.from_dict(predicted, {"words": token_ref, "tags": tags_ref})
print( 'predicted', example.predicted )
print( 'reference', example.reference )

predicted Apply some sunscreen 
reference Apply some sun screen 


In [ ]:

count = collections.defaultdict(int)

for plaintext, in random.sample( bwb_rows, 10):

    for paragraph in re.split( '\n{2,}', plaintext ):
        doc = english( paragraph )
        #print( paragraph )
        for chunk in doc.noun_chunks:
            #print( 'NC[%r]'%chunk )
            count[chunk.text] += 1

it = list(count.items())
it.sort(key = lambda x:x[1])
for s, cnt in it:
    print('%s\t%s'%(cnt,s))